# Artifical Neural Network

In [2]:
import numpy as np
import matplotlib.pyplot as plt 
import pkbar
import pandas

In [25]:
class Parameter:
    def __init__(self,tensor):
        self.weights = tensor 
        self.gradients = np.zeros_like(self.weights)
        self.bias = np.zeros((tensor.shape[-1]))
        self.bias_gradients = np.zeros_like(self.bias)

class Layer:
    def __init__(self):
        self.parameters = None 
    def init_param(self, tensor):
        param = Parameter(tensor)
        self.parameters = param 
        return param 
    def update(self, optimizer):
        optimizer.update(self.parameters)

class SGD:
    def __init__(self,lr=0.1):
        self.lr = lr 
    def update(self,param):
        param.weights -= self.lr * param.gradients
        param.bias -= self.lr * param.bias
class Sigmoid(Layer):
    def __init__(self):
        self.parameters = None 

    def backward(self,D,X):
        S = 1 / (1+np.exp(-X))
        return D * (S*(1-S))
    def forward(self,X):
        return 1/(1+np.exp(-X))
    def __str__(self) -> str:
        return "Sigmoid Layer"
class Softmax(Layer):
    def __init__(self):
        self.parameters = None 
    def backward(self,D,X):
        return D 
    def softmax(self,X):
        k = np.sum(np.exp(X),axis=-1).reshape(-1,1)
        return np.exp(X) / k
    def forward(self,X):
        return self.softmax(X)
    def __str__(self) -> str:
        return "Softmax Layer"
class Tanh(Layer):
    def __init__(self):
        self.parameters = None 
    def backward(self,D,X):
        return D * (1-(np.tanh(X)*np.tanh(X)))
    def forward(self,X):
        return np.tanh(X)
    def __str__(self) -> str:
        return "Tanh Layer"
class ReLU(Layer):
    def __init__(self):
        self.parameters = None 
    def backward(self, D,X):
        return D * (1*(X>0))
    def forward(self,X):
        return np.maximum(0,X)
    def __str__(self) -> str:
        return "ReLU Layer"
class Linear(Layer):
    '''
    Linear layer
    '''
    def __init__(self,inputs,outputs) -> None:
        super().__init__()
        tensor = np.random.randn(inputs,outputs)
        self.parametres = self.init_param(tensor)
    def backward(self, D,X):
        self.parametres.gradients = (X.T@ D)
        X_bias = np.ones(X.shape[0])
        self.parametres.bias_gradients = X_bias.T@D 
        # D * w.T
        return D @ self.parametres.weights.T
    def forward(self, X):
        # X*w * b
        return X@self.parametres.weights + self.parametres.bias
    def __str__(self) -> str:
        return "Linear Layer"    

In [33]:
class Model():
    def __init__(self):
        self.computational_graph = []
    def add(self,layer):
        self.computational_graph.append(layer)
    def compiler(self,loss,optimizer):
        self.loss = loss
        self.optimizer = optimizer 
    def forward(self,X):
        Y_int = X 
        Y_int_list = [] 
        for layer in self.computational_graph:
            Y_int_list.append(Y_int)
            Y_ = layer.forward(Y_int)
            Y_int = Y_ 
        return Y_, Y_int_list
    def fit_batch(self,X,Y):
        out = X
        Y_, Y_int_list = self.forward(X)
        print(Y_,'\n',Y)
        L,D = self.loss(Y_,Y)
        for Y_int, layer in zip(Y_int_list[::-1],self.computational_graph[::-1]):
            print(layer)
            D = layer.backward(D,Y_int)

            if layer.parametres is not None:
                layer.update(self.optimizer)
        return L 
    def fit(self, X,Y,epochs, bs):
        losses = []
        pbar = pkbar.Pbar(name='Training....', target= epochs)
        kbar = pkbar.Kbar(target=epochs)
        for epoch in range(epochs):
            loss = 0.0 
            for i in range(0,len(X),bs):
                loss += self.fit_batch(X[i:i+bs],Y[i:i+bs])
            losses.append(loss)
            kbar.update(epoch+1, values=[("loss",loss)])
        return losses

In [41]:
def bce_loss(y_pred,y_true):
    loss = (y_true*np.log(y_pred))
    print(len(y_pred))
    return -np.sum(loss)/len(y_pred)

## XOR Gate

In [42]:
X = np.array([[0,0],[0,1],[1,0],[1,1]],dtype=float)
Y = np.array([[0],[1],[1],[0]],dtype=float)


EPOCHS = 10000

model = Model()
model.add(Linear(2,10))
model.add(Sigmoid())

model.add(Linear(10,1))
model.add(Sigmoid())


model.compiler(bce_loss, SGD(lr=5e-1))
losses = model.fit(X,Y, epochs=EPOCHS, bs = X.shape[0])
plt.plot(range(1, EPOCHS+1),losses)
plt.ylabel('BCE')
plt.xlabel('Number of epochs')

plt.show()
print(losses[-1])

Training....
[[0.37826165]
 [0.30580289]
 [0.26832181]
 [0.24180999]] 
 [[0.]
 [1.]
 [1.]
 [0.]]
4


TypeError: cannot unpack non-iterable numpy.float64 object